<a href="https://colab.research.google.com/github/natecross123/HeartfailureAI-model/blob/main/Heartfailure_Kaggle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q Kaggle

In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"nathancrossdale","key":"ce8057c7155839faf50afd25943f58b3"}'}

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/


In [ ]:
! chmod 600 ~/.kaggle/kaggle.json


In [ ]:
!kaggle competitions download -c heart-failure-prediction-comp-3220-summer-2025

  0% 0.00/11.5k [00:00<?, ?B/s]
100% 11.5k/11.5k [00:00<00:00, 53.0MB/s]


In [ ]:
!unzip heart-failure-prediction-comp-3220-summer-2025.zip


Archive:  heart-failure-prediction-comp-3220-summer-2025.zip
  inflating: heart_test.csv          
  inflating: heart_train.csv         
  inflating: sample_submission.csv   


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [ ]:
df_train = pd.read_csv('heart_train.csv')
print(df_train.head())
print(df_train.isnull().sum())


   PatientId  Age Sex ChestPainType  RestingBP  Cholesterol  FastingBS  \
0          0   34   M           ATA        150          214          0   
1          1   59   M           ASY        178            0          1   
2          2   58   M           ASY        115            0          1   
3          3   60   M           ASY        130          253          0   
4          4   52   M           ASY        165            0          1   

  RestingECG  MaxHR ExerciseAngina  Oldpeak ST_Slope  HeartDisease  
0         ST    168              N      0.0       Up             0  
1        LVH    120              Y      0.0     Flat             1  
2     Normal    138              N      0.5       Up             1  
3     Normal    144              Y      1.4       Up             1  
4     Normal    122              Y      1.0       Up             1  
PatientId         0
Age               0
Sex               0
ChestPainType     0
RestingBP         0
Cholesterol       0
FastingBS         0
R

In [ ]:
# Check for and remove any null values
df_train.dropna(inplace=True)
print(df_train.isnull().sum())

PatientId         0
Age               0
Sex               0
ChestPainType     0
RestingBP         0
Cholesterol       0
FastingBS         0
RestingECG        0
MaxHR             0
ExerciseAngina    0
Oldpeak           0
ST_Slope          0
HeartDisease      0
dtype: int64


In [ ]:
from sklearn.preprocessing import StandardScaler
numeric_cols = ['Age', 'RestingBP', 'Cholesterol', 'MaxHR', 'Oldpeak']
scaler = StandardScaler()
df_train[numeric_cols] = scaler.fit_transform(df_train[numeric_cols])
print(df_train[numeric_cols].head())

        Age  RestingBP  Cholesterol     MaxHR   Oldpeak
0 -2.056345   0.936570     0.148419  1.206471 -0.823144
1  0.578343   2.435605    -1.831850 -0.669009 -0.823144
2  0.472955  -0.937224    -1.831850  0.034296 -0.354837
3  0.683730  -0.134170     0.509310  0.268731  0.488116
4 -0.159370   1.739625    -1.831850 -0.590864  0.113470


In [ ]:
# Convert categorical columns to numerical using one-hot encoding
df_train = pd.get_dummies(df_train, columns=['Sex', 'ChestPainType', 'RestingECG', 'ExerciseAngina', 'ST_Slope'], drop_first=True)
print(df_train.head())

   PatientId       Age  RestingBP  Cholesterol  FastingBS     MaxHR   Oldpeak  \
0          0 -2.056345   0.936570     0.148419          0  1.206471 -0.823144   
1          1  0.578343   2.435605    -1.831850          1 -0.669009 -0.823144   
2          2  0.472955  -0.937224    -1.831850          1  0.034296 -0.354837   
3          3  0.683730  -0.134170     0.509310          0  0.268731  0.488116   
4          4 -0.159370   1.739625    -1.831850          1 -0.590864  0.113470   

   HeartDisease  Sex_M  ChestPainType_ATA  ChestPainType_NAP  \
0             0   True               True              False   
1             1   True              False              False   
2             1   True              False              False   
3             1   True              False              False   
4             1   True              False              False   

   ChestPainType_TA  RestingECG_Normal  RestingECG_ST  ExerciseAngina_Y  \
0             False              False           True

In [ ]:
X = df_train.drop('HeartDisease', axis=1)
y = df_train['HeartDisease']
print(X.head())
print(y.head())

   PatientId       Age  RestingBP  Cholesterol  FastingBS     MaxHR   Oldpeak  \
0          0 -2.056345   0.936570     0.148419          0  1.206471 -0.823144   
1          1  0.578343   2.435605    -1.831850          1 -0.669009 -0.823144   
2          2  0.472955  -0.937224    -1.831850          1  0.034296 -0.354837   
3          3  0.683730  -0.134170     0.509310          0  0.268731  0.488116   
4          4 -0.159370   1.739625    -1.831850          1 -0.590864  0.113470   

   Sex_M  ChestPainType_ATA  ChestPainType_NAP  ChestPainType_TA  \
0   True               True              False             False   
1   True              False              False             False   
2   True              False              False             False   
3   True              False              False             False   
4   True              False              False             False   

   RestingECG_Normal  RestingECG_ST  ExerciseAngina_Y  ST_Slope_Flat  \
0              False           T

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

log_reg_model = LogisticRegression(max_iter=1100)
log_reg_model.fit(X_train, y_train)

LogisticRegression(max_iter=1100)

In [ ]:
y_pred_lr = log_reg_model.predict(X_test)
accuracy_lr = accuracy_score(y_test, y_pred_lr)
print(f"Logistic Regression Test Accuracy: {accuracy_lr:.2%}")

Logistic Regression Test Accuracy: 85.98%


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Define the simple neural network model
nn_model = Sequential([
    Dense(32, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(1, activation='sigmoid')
])

nn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
nn_model.fit(X_train, y_train, epochs=50, batch_size=32, verbose=0)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:

df_test = pd.read_csv('heart_test.csv')
df_test[numeric_cols] = scaler.transform(df_test[numeric_cols])

# One-hot encode categorical columns
df_test = pd.get_dummies(df_test, columns=['Sex', 'ChestPainType', 'RestingECG', 'ExerciseAngina', 'ST_Slope'], drop_first=True)

missing_cols = set(X_train.columns) - set(df_test.columns)
for c in missing_cols:
    df_test[c] = 0

df_test = df_test[X_train.columns]

# Make predictions using the Random Forest model (as it had the highest accuracy in the notebook)
test_predictions = rf_model.predict(df_test)

# Create the submission file
submission_df = pd.DataFrame({'PatientId': df_test['PatientId'], 'HeartDisease': test_predictions})

# Save the submission file as a CSV
submission_df.to_csv('submission.csv', index=False)

print("Submission file created successfully!")

Submission file created successfully!


In [ ]:
loss_nn, accuracy_nn = nn_model.evaluate(X_test, y_test, verbose=0)
print(f"Simple Neural Network Test Accuracy: {accuracy_nn:.2%}")

Simple Neural Network Test Accuracy: 84.76%


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Define the Random Forest model
rf_model = RandomForestClassifier(random_state=42)


rf_model.fit(X_train, y_train)

y_pred_rf = rf_model.predict(X_test)

accuracy_rf = accuracy_score(y_test, y_pred_rf)
print(f"Random Forest Test Accuracy: {accuracy_rf:.2%}")

Random Forest Test Accuracy: 86.59%


In [ ]:
!kaggle competitions submit -c heart-failure-prediction-comp-3220-summer-2025 -f submission.csv -m " submission"

100% 623/623 [00:00<00:00, 961B/s]
Successfully submitted to Heart Failure Prediction COMP3220 Summer 2025